In [1]:
import sys
sys.path.append('C:\\Anaconda2\\Lib\\site-packages\\blpfunctions')
from blpfunctions import blpfunctions as blp
#import blpfunctions as blp
import datetime as dt
import pandas as pd
from pandas.tseries.offsets import *
from holidays_jp import CountryHolidays
import dateutil.rrule as RR
import sys
import csv
import numpy as np
import time
import gc
import itertools
%matplotlib inline

In [2]:
def _load_data(data, n_prev = 62):  
    """
    data should be pd.DataFrame()
    """
    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data.iloc[i:i+n_prev-1].as_matrix())
        docY.append(data.iloc[i+n_prev].as_matrix())
        #docX.append(data.iloc[i:i+n_prev])
        #docY.append(data.iloc[i+n_prev])
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

def train_test_split(df, test_size=0.082, n_prev = 62):  
    """
    This just splits data to training and testing parts
    """ 
    ntrn = round(len(df) * (1 - test_size),0)
    ntrn = int(ntrn)
    X_train, y_train = _load_data(df.iloc[0:ntrn], n_prev)
    X_test, y_test = _load_data(df.iloc[ntrn:], n_prev)

    return X_train, y_train, X_test, y_test

In [3]:
def proc_data(stock_list, window):      
    fld_l = ["VOLUME"]
    event_l = ["TRADE"]
    iv = 5
    ed1 = "2017-09-25T15:00:00"
    days = 60
    test_size = 0.115
    Train_list = []
    test_list =[]
    y_dict = {}
    y_test_dict = {}
    for ind,stock in enumerate(stock_list):
        avc, vc = horiz_stock_volcurve([stock +' JT Equity'],event_l,ed1,days,iv,fld_l)
        X_train, y_train, X_test, y_test = train_test_split(vc[[stock]], test_size, window)
        aX_train, ay_train, aX_test, ay_test = train_test_split(avc[[stock]], test_size, window)
        Train_list.append(X_train)
        Train_list.append(aX_train)
        y_dict[stock] = y_train
        #y_list.append(ay_train)
        test_list.append(X_test)  
        test_list.append(aX_test) 
        y_test_dict[stock]=y_test
        #y_test_list.append(ay_test)
    return Train_list, y_dict, test_list, y_test_dict

In [4]:
def horiz_stock_volcurve(ind, event, edate, numdays, interval,fld_lst):
    #sec_list = blp.get_index(ind)
    #sec_list = ['5801 JT Equity']
    sec_list = ind
    volcurves = pd.DataFrame()
    fmt = "%Y-%m-%d" + 'T' + "%H:%M:%S"  #Assumes no milliseconds
    endDateTime = dt.datetime.strptime(edate, fmt)
    #skip SQ and holidays (Actually turns out cannot skip SQ days in blp call)
    day1 = dt.datetime(int(edate[0:4]),1,1)
    sq = list(RR.rrule(RR.MONTHLY,byweekday=RR.FR,bysetpos=2,dtstart=day1,until=endDateTime))
    hols = list(zip(*CountryHolidays.get('JP', int(edate[0:4])))[0])
    skipdays = hols #+ sq if want to skip SQ days
    bday_jp = CustomBusinessDay(holidays=skipdays)
    startDateTime = endDateTime.replace(hour=9) - numdays*bday_jp
    numdays = pd.date_range(startDateTime, endDateTime, freq=bday_jp).nunique()
    sdate = startDateTime.strftime(fmt)
    vc = pd.DataFrame()
    avc = pd.DataFrame()
    for stock in sec_list:
        volcurves=blp.get_Bars(stock, event, sdate, edate, interval, fld_lst)
    volcurves.rename(columns={'VOLUME':stock},inplace=True)
    #process the raw data into historical averages
    volcurves.rename(columns=lambda x: x[:4], inplace=True)
    timevect = pd.Series(volcurves.index.values)
    timeframet = timevect.to_frame()
    timeframet.columns = ['date']
    timeframet.set_index(timevect,inplace=True)
    timeframet['bucket'] = timeframet['date'].apply(lambda x: dt.datetime.strftime(x, '%H:%M:%S'))
    timeframet['date'] = timeframet['date'].apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))  
    absvolcurve=timeframet.join(volcurves).fillna(0).reset_index(level=0,drop=True)
    absvolcurve = absvolcurve[absvolcurve.bucket != '15:10:00']
    absvolcurve['bucket2'] = absvolcurve['date'] + absvolcurve['bucket']
    buckets = sorted(absvolcurve['bucket'].unique())
    dates = sorted(absvolcurve['date'].unique())
    df = pd.DataFrame([dat+buk for dat in dates for buk in buckets],columns=['bucket2'])
    absvolcurve = df.merge(absvolcurve, how= 'left', on='bucket2').fillna(0).reset_index(level=0,drop=True) 
    volcurve = absvolcurve.copy()
    sums = absvolcurve.groupby('date')[stock[:4]].sum()
    absvolcurve['sums'] = absvolcurve['date'].map(sums)
    volcurve['ratio'] = absvolcurve[stock[:4]]/absvolcurve['sums']
    volcurve['sumcum'] = volcurve.groupby('date')['ratio'].cumsum()
    absvolcurve = absvolcurve.drop(['sums','bucket2','date','bucket'],axis=1)
    volcurve = volcurve.drop(['ratio',stock[:4],'bucket2','date','bucket'],axis=1)
    volcurve = volcurve.rename(columns = {'sumcum':stock[:4]})     
    volcurve = volcurve.interpolate()
    avc[stock[:4]] = absvolcurve[stock[:4]]
    vc[stock[:4]] = volcurve[stock[:4]]
            
    return avc.fillna(method='bfill'), vc.fillna(method='bfill')
    #return absvolcurve.fillna(method='bfill'), volcurve.fillna(method='bfill')

In [ ]:
X_train.shape

In [ ]:
fld_lst = ["VOLUME"]
ind = ["NKY"]
event = ["TRADE"]
interval = 5
edate = "2017-09-25T15:00:00"
numdays = 60
avc, vc = horiz_stock_volcurve(ind, event, edate, numdays, interval,fld_lst)
window = 60
length_of_sequences=60
feats = 1
X_train, y_train, X_test, y_test = train_test_split(vc[['5801']], n_prev = length_of_sequences)
aX_train, ay_train, aX_test, ay_test = train_test_split(avc[['5801']], n_prev = length_of_sequences)


In [ ]:
model = build_BDLSTM_model([feats,window,1])
model.fit([np.array(X_train), np.array(aX_train)], np.array(y_train), batch_size=5, epochs=50, validation_split=0.1, verbose=1)

In [ ]:
#p = model.predict([X_test,aX_test],batch_size=50)
train_mse, test_mse = BDLSTM_model_score(model, X_train, y_train, aX_train, aX_test, X_test, y_test) 

In [ ]:
print train_mse, test_mse 

In [53]:
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Activation, Input, LSTM, Dropout, Bidirectional, Lambda, merge
from keras.layers.core import Flatten, Permute, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from attention_lstm import AttentionLSTMWrapper

early_stopping = EarlyStopping(monitor='val_loss', patience=15)
SINGLE_ATTENTION_VECTOR = False

def attention_3d_block(inputs,shape):
    #inputs.shape = (batch_size, time_steps, input_dim)
    #input_dim = int(inputs.shape[2])
    input_dim = shape[1]
    TIME_STEPS=shape[0]
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    return output_attention_mul

def build_BDLSTM_model(layers):
    d = 0.0
    num_lstm = 128
    
    shape = [layers[1], layers[0]]
      
    shared_lstm = LSTM(num_lstm, dropout=d, recurrent_dropout=d, return_sequences=True, consume_less='mem')
    
    s1 = Input(shape=shape)
    s2 = Input(shape=shape)
    
    attention_mul1 = attention_3d_block(s1,shape)
 
    
    x1 = shared_lstm(attention_mul1)
    #x1 = BatchNormalization()(x1)
    x2 = LSTM(num_lstm, dropout=d, recurrent_dropout=d, return_sequences=True, consume_less='mem')(s2)
    x2 = BatchNormalization()(x2)
       
    merged = concatenate([x1,x2])
    
    merged = Dense(32, kernel_initializer="uniform", activation='relu')(merged)
    merged = Dropout(0.6)(merged)
    
    merged = Dense(32, kernel_initializer="uniform", activation='relu')(merged)
    merged = Dropout(0.6)(merged)
    
    preds = Dense(layers[0], kernel_initializer="uniform", activation='linear')(merged)
        
    start = time.time()
    model = Model(inputs = [s1,s2], outputs = [preds])
    model.compile(loss='mse',optimizer='nadam', metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

def BDLSTM_model_score(model, X_train, y_train, aX_train, aX_test, X_test, y_test):
    trainScore = model.evaluate([X_train,aX_train], y_train ,batch_size=50, verbose=0)
    #print('Train Score: %.5f MSE (%.4f RMSE)' % (trainScore[0], np.sqrt(trainScore[0])))

    testScore = model.evaluate([X_test, aX_test], y_test, batch_size=50, verbose=0)
    #print('Test Score: %.5f MSE (%.4f RMSE)' % (testScore[0], np.sqrt(testScore[0])))
    return trainScore[0], testScore[0]

In [ ]:
model.summary()

In [ ]:
window = 3
feats = len(relcurve.columns)
X_train, y_train, X_test, y_test = load_data(relcurve, window)
aX_train, ay_train, aX_test, ay_test = load_data(abscurve, window)

model = build_model([feats,window,1])
model.fit([X_train, aX_train], y_train, batch_size=5, epochs=200, validation_split=0.1, verbose=1)

In [ ]:
# print(X_test[-1])
#diff=[]
#ratio=[]
p = model.predict([X_test, aX_test],batch_size=4096)
print (p.shape)
# for each data index in test data
# for u in range(len(y_test)):
#     # pr = prediction day u
#     pr = p[u][0]
#     # (y_test day u / pr) - 1
#     ratio.append((y_test[u]/pr)-1)
#     diff.append(abs(y_test[u]- pr))
#     # print(u, y_test[u], pr, (y_test[u]/pr)-1, abs(y_test[u]- pr))
#     # Last day prediction
#     # print(p[-1]) 

In [ ]:
import matplotlib.pyplot as plt2
#p = model.predict([X_test, aX_test],batch_size=4096)
plt2.plot(p,color='red', label='Prediction')
plt2.plot(y_test,color='blue', label='Actual')
plt2.legend(loc='best')
plt2.show()

In [ ]:
X_train.shape

In [ ]:
#Build, Train, and Save results per Cluster

In [ ]:
#Cluster Cross-Stock Bidirectional LSTM
import keras.backend as K
from keras.models import Model, Input
from keras.layers import Activation, Input, LSTM, Dropout, Bidirectional, Lambda
from keras.layers.core import Dense, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=20)

def build_cluster_cross_stock_model(layers):
    d = 0.3
    num_lstm = 256
    shape = [layers[1], layers[0]]
    
    lstm = LSTM(num_lstm, recurrent_dropout=d)
    #lstm = LSTM(num_lstm, dropout=d)
    batchnorm = BatchNormalization()
    
    input_list = []
    for num in range(0,2*layers[2]):
        input_list.append(Input(shape=shape))
    print('Using {} tensors as input.'.format(len(input_list)))    
        
    lstm_layers=[]
    for i in range(0,layers[2]):
        lstm_layers.append(lstm)
    print('Using {} interlaced LSTM layers on those inputs.'.format(len(lstm_layers)))
    
    lstm_outputs=[]
    for i in range(1,2*layers[2],2):
        id_lstm_layer = i//2
        shared_lstm = lstm_layers[id_lstm_layer]
        lstm_outputs.append(shared_lstm(input_list[i - 1]))
        lstm_outputs.append(batchnorm(shared_lstm(input_list[i])))
        print('LSTM layer id = {} is linked to inputs whose ids are {} and {}.'.format(id_lstm_layer, i - 1, i))
                            
    merged = Lambda(lambda x: K.stack(x,axis=1),output_shape=(2*layers[2], 2*num_lstm))(lstm_outputs)
    merged = Flatten()(merged)                        
    merged = Dense(128, kernel_initializer="uniform", activation='relu')(merged)
    merged = Dropout(0.7)(merged)
    merged = Dense(128, kernel_initializer="uniform", activation='relu')(merged)
    merged = Dropout(0.7)(merged)
    preds = Dense(layers[0], kernel_initializer="uniform", activation='linear')(merged)
    
    start = time.time()
    model = Model(inputs = input_list, outputs = [preds])
    model.compile(loss='mse',optimizer='nadam', metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

def cross_stock_model_score(model, Train, y, test, test_y):
    trainScore = model.evaluate(Train, y ,batch_size=50, verbose=0)
    #print('Train Score: %.5f MSE (%.4f RMSE)' % (trainScore[0], np.sqrt(trainScore[0])))

    testScore = model.evaluate(test, test_y, batch_size=50, verbose=0)
    #print('Test Score: %.5f MSE (%.4f RMSE)' % (testScore[0], np.sqrt(testScore[0])))
    return trainScore[0], testScore[0]


In [55]:
def go(clust,mode):
    
    window = 62
    feats = 1
    
    print('Training in %s mode' % mode)
    stock_list = asses.item()[clust]
    output = pd.DataFrame(columns=['stock','bst_val_score','train_rmse','test_rmse'],index=stock_list)
    output['stock'] = stock_list
    num_stocks = len(stock_list)
    #tb = TensorBoard(log_dir='./tblogs', histogram_freq=0, write_graph=True, write_grads=False, write_images=False)
    
    Train_list, y_dict, test_list, y_test_dict = proc_data(stock_list, window)
    
    if mode == 'cross':
        model = build_cluster_cross_stock_model([feats,window-1,num_stocks])
        STAMP = 'Horiz_Cross-stock-cluster-' + str(clust)
        bst_model_path = STAMP + '.h5'
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
        
    elif mode == 'single':
        model = build_BDLSTM_model([feats,window-1,num_stocks])
        model.summary()
        STAMP = 'Horiz_BDLSTM_61_10_50-cluster-' + str(clust)
        bst_model_path = STAMP + '.h5'
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
     
    for ind,stock in enumerate(stock_list): 
        if mode == 'cross':
            model.load_weights(bst_model_path)
            hist = model.fit(Train_list, y_dict[stock], batch_size=61, epochs=100, validation_split=0.0943, verbose=True,
                  callbacks=[early_stopping])
            bst_val_score=min(hist.history['val_loss'])
            #p = model.predict(test_list,batch_size=50)
            train_mse, test_mse = cross_stock_model_score(model, Train_list, y_dict[stock], test_list, y_test_dict[stock])
            
        elif mode == 'single':
            X_train = Train_list[2*ind]
            aX_train = Train_list[2*ind+1]           
            X_test = test_list[2*ind]
            aX_test = test_list[2*ind+1]
            y_train = y_dict[stock]
            y_test = y_test_dict[stock]
            #model.load_weights(bst_model_path)
            hist = model.fit([X_train, aX_train], y_train, batch_size=61, epochs=100, validation_split=0.0943, verbose=True,
                  callbacks=[early_stopping])
            bst_val_score=min(hist.history['val_loss'])
            #p = model.predict([X_test,aX_test],batch_size=50)
            train_mse, test_mse = BDLSTM_model_score(model, X_train, y_train, aX_train, aX_test, X_test, y_test)     
      
        output['bst_val_score'][stock] = bst_val_score
        output['train_rmse'][stock] = np.sqrt(train_mse)
        output['test_rmse'][stock] = np.sqrt(test_mse)
    
        print('Stock %s: Best Val Score: %.4f, Train Score: %.4f RMSE, Test Score %.4f RMSE' 
              % (stock[:4], bst_val_score, np.sqrt(train_mse), np.sqrt(test_mse)))
        
    model.save_weights(bst_model_path)
    output.to_csv(STAMP + '-Horizontal.csv.gz', index=False, compression='gzip')

In [56]:
def run(clusters):
    #test on smallest cluster
    clusters =[4]
    mode = 'single'
    #mode = 'cross'
    
    for clust in clusters:
        go(clust,mode)
    

In [57]:
if __name__ == '__main__':
    asses = np.load('clusters.npy')
    clusters = asses.item().keys()
    run(clusters)

Training in single mode


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:34: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, implementation=1, return_sequences=True, dropout=0.0, recurrent_dropout=0.0)`
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:25: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:44: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, implementation=1, return_sequences=True, dropout=0.0, recurrent_dropout=0.0)`


('Compilation Time : ', 0.00800013542175293)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_49 (InputLayer)            (None, 61, 1)         0                                            
____________________________________________________________________________________________________
permute_12 (Permute)             (None, 1, 61)         0           input_49[0][0]                   
____________________________________________________________________________________________________
reshape_10 (Reshape)             (None, 1, 61)         0           permute_12[0][0]                 
____________________________________________________________________________________________________
dense_23 (Dense)                 (None, 1, 61)         3782        reshape_10[0][0]                 
______________________________________________

ValueError: Error when checking target: expected dense_26 to have 3 dimensions, but got array with shape (3231L, 1L)

In [ ]:
fld_l = ["VOLUME"]
event_l = ["TRADE"]
iv = 5
ed1 = "2017-09-25T15:00:00"
days = 60
#abscurve, relcurve = horiz_stock_volcurve(['5801 JT Equity'],event_l,ed1,days,iv,fld_l)
X_train, y_train, X_test, y_test = train_test_split(relcurve, 0.115, 62)
aX_train, ay_train, aX_test, ay_test = train_test_split(abscurve, 0.115, 62)

In [ ]:
X_train.shape

In [ ]:
len(X_test)

In [ ]:
len(abscurve['bucket'].unique())

In [ ]:
model = build_BDLSTM_model([3,61,1])
model.fit([X_train, aX_train], y_train, batch_size=5, epochs=1, validation_split=0.1, verbose=1)

In [ ]:
model.summary()

In [ ]:
import theano 
theano.__version__

In [44]:
model.summary()

NameError: name 'model' is not defined